In [96]:
import requests, json
import pandas as pd
from pandas_profiling import ProfileReport
from pprint import pprint
from bs4 import BeautifulSoup as bs
import datetime
import pickle

In [97]:
params = {
    'area': [113, 40, 1001],
    'text': 'game* OR гейм*',
    'search_field': ['description',],
    'employment': ['full', 'part'],
    'schedule': ['remote',],
    # 'professional_role': [25, 96],
    'date_from': '2022-06-01',
    'per_page': 100,
}

In [98]:
# https://kaliningrad.hh.ru/article/1175#simple-search
def get_vacancies(paramsб, page=0):
    params['page'] = page
    return requests.get('https://api.hh.ru/vacancies', params=params)

In [99]:
vacancies = get_vacancies(params)
print(f"Pages: {vacancies.json()['pages']}")

Pages: 7


In [100]:
def get_df(items):
    df = pd.DataFrame.from_records(items)
    df.drop(columns=[
                    'has_test',
                    'premium',
                    'address',
                    'department',
                    'response_letter_required',
                    'sort_point_distance',
                    'response_url',
                    'relations',
                    'schedule',
                    'working_days',
                    'working_time_intervals',
                    'working_time_modes',
                    'accept_temporary',
                    'apply_alternate_url',
                    'insider_interview',
                    'adv_response_url',
                    'contacts',    
                ], inplace=True)
    df['area'] = df['area'].apply(lambda x: x.get('name'))
    df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
    df['type'] = df['type'].apply(lambda x: x.get('name'))
    df['employer'] = df['employer'].apply(lambda x: x.get('name'))
    df = pd.concat([df.drop(['snippet'], axis=1), df['snippet'].apply(pd.Series)], axis=1)
    return df

In [101]:
df = pd.DataFrame()
for page in range(vacancies.json()['pages']):
    v = get_vacancies(params, page=page)
    d = get_df(v.json()['items'])
    df = df.append(d, ignore_index=True)

/tmp/ipykernel_1044069/980802284.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_1044069/980802284.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_1044069/980802284.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_1044069/980802284.py:23: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a fut

In [102]:
deep_vac = []
for i in df.itertuples():
    if i.url:
        r = requests.get(i.url)
        deep_vac.append(r.json())
deep_df = pd.DataFrame.from_records(deep_vac)

In [103]:
def get_deep_df(df):
    df = df.drop(columns=[
        'premium',
        'billing_type',
        'relations',
        'accept_handicapped',
        'insider_interview',
        'response_letter_required',
        'type',
        'address',
        'allow_messages',
        'schedule',
        'department',
        'contacts',
        'branded_description',
        'vacancy_constructor_template',
        'accept_kids',
        'archived',
        'response_url',
        'code',
        'quick_responses_allowed',
        'driver_license_types',
        'accept_incomplete_resumes',
        'initial_created_at',
        'negotiations_url',
        'suitable_resumes_url',
        'has_test',
        'test',
        'working_days',
        'working_time_intervals',
        'working_time_modes',
        'accept_temporary',
        'hidden',
                ])
    
    for name in ['area', 'employer', 'experience', 'employment']:
        df[name] = df[name].apply(lambda x: x.get('name'))
    df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
    
    def skills_to_list(x):
        if x:
            if isinstance(x[0], dict):
                items = [i['name'] for i in x]
                return items
            else:
                return x

    for name in ['key_skills', 'specializations', 'professional_roles']:
        df[name] = df[name].apply(skills_to_list)
        
    df['languages'] = df['languages'].apply(lambda x: x if x else None)
    
    def get_text(x):
        if x:
            soup = bs(x, features="html.parser")
            text = soup.get_text()
            return text
        
    df['description'] = df['description'].apply(get_text)
    
    return df

In [104]:
deep_df1 = get_deep_df(deep_df)

/tmp/ipykernel_1044069/2199928276.py:38: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_1044069/2199928276.py:38: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_1044069/2199928276.py:38: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df = pd.concat([df.drop(['salary'], axis=1), df['salary'].apply(pd.Series)], axis=1)
/tmp/ipykernel_1044069/2199928276.py:38: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a

In [105]:
deep_df1.sort_values(by=['employer'], inplace=True)
deep_df1.drop(
    ['employment', 'specializations', 'published_at', 
     'created_at', 'apply_alternate_url', 'gross', 'professional_roles'], 
    axis=1, 
    inplace=True
    )
# deep_df1.drop(deep_df1[deep_df1['languages'].notnull()].index, inplace=True)
# deep_df1.drop(['languages'], axis=1, inplace=True)
deep_df1.sort_values(by=['employer'], inplace=True)

In [106]:
k_skills = set()
for i in deep_df1['key_skills']:
    if i:
        k_skills.update(list(i))

In [107]:
k_skills

{'.NET',
 '.NET Core',
 '.NET Developer',
 '.NET Framework',
 '1C',
 '1С: Комплексная автоматизация',
 '1С: Предприятие 8',
 '1С: Торговля',
 '1С: Торговля и склад',
 '1С: Управление Торговлей',
 '2D',
 '2D Art',
 '2D Artist',
 '2D art',
 '2D анимация',
 '2d artist',
 '3D',
 '3D Max',
 '3D Studio Max',
 '3D Моделирование',
 '3D-графика',
 '3ds Max',
 'A/B тесты',
 'AAA project',
 'AI',
 'API',
 'ARPU',
 'ASA',
 'ASP.NET',
 'ASP.NET Core',
 'ATL',
 'AWS',
 'Account Management',
 'Accounting',
 'Adjust',
 'Adobe',
 'Adobe After Effect',
 'Adobe Animate',
 'Adobe CS',
 'Adobe Design',
 'Adobe Flash',
 'Adobe Illustrator',
 'Adobe InDesign',
 'Adobe Photoshop',
 'Adobe Premier Pro',
 'AdsKeeper',
 'Aerospike',
 'After Effects',
 'Agile',
 'Agile Project Management',
 'Alertmanager',
 'Analysis',
 'Analytical skills',
 'Android',
 'Angular',
 'AngularJS',
 'AnimBP',
 'Animation',
 'Ansible',
 'Apache Maven',
 'Applovin',
 'AppsFlyer',
 'Appsflyer',
 'Art Lead',
 'Artist',
 'Asana',
 'Asteri

In [108]:
deep_df1.drop_duplicates(subset=['description'], inplace=True)
deep_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 453 entries, 503 to 57
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             453 non-null    object 
 1   name           453 non-null    object 
 2   area           453 non-null    object 
 3   experience     453 non-null    object 
 4   description    453 non-null    object 
 5   key_skills     378 non-null    object 
 6   employer       453 non-null    object 
 7   alternate_url  453 non-null    object 
 8   languages      107 non-null    object 
 9   from           120 non-null    float64
 10  to             81 non-null     float64
 11  currency       135 non-null    object 
dtypes: float64(2), object(10)
memory usage: 46.0+ KB


In [109]:
deep_df1

,id,name,area,experience,description,key_skills,employer,alternate_url,languages,from,to,currency
503,76217189,Unreal Engine/С++ Developer,Москва,От 3 до 6 лет,Обязанности: Реализация игровых механик Ре...,"[Работа в команде, Ведение документации, Удале...",2040 World,https://hh.ru/vacancy/76217189,"[{'id': 'eng', 'name': 'Английский', 'level': ...",NaN,NaN,NaN
502,76219542,Lead Unreal Engine developer,Москва,От 3 до 6 лет,Команда 2040WORLD в связи с активным развитием...,"[C#, Git, Unreal Engine]",2040 World,https://hh.ru/vacancy/76219542,"[{'id': 'eng', 'name': 'Английский', 'level': ...",NaN,NaN,NaN
363,76755566,Методист,Москва,От 1 года до 3 лет,Приветствуем! Мы 3D Club – ведущая онлайн-школ...,None,3DCLUB,https://hh.ru/vacancy/76755566,None,50000.0,NaN,RUR
479,76382277,GameDev-эксперт,Москва,От 1 года до 3 лет,Приветствуем! Мы 3D Club – ведущая онлайн-школ...,"[Autodesk 3ds Max, 3ds Max, 3D Моделирование, ...",3DCLUB,https://hh.ru/vacancy/76382277,None,NaN,NaN,NaN
347,76787262,Community Manager / SMM Manager,Москва,От 1 года до 3 лет,A+ Props основана в 2014 году. Мы делаем игры ...,"[Английский язык, Грамотность, Навыки межлично...",A+ Props,https://hh.ru/vacancy/76787262,"[{'id': 'eng', 'name': 'Английский', 'level': ...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
25,77220808,Support specialist (gamedev),Иваново (Ивановская область),От 1 года до 3 лет,Привет! Наша компания занимается разработкой м...,"[Английский язык, Грамотная речь, Работа с люд...",ЭФ ИКС ГЕЙМС,https://hh.ru/vacancy/77220808,"[{'id': 'eng', 'name': 'Английский', 'level': ...",50000.0,NaN,RUR
496,76283384,Дизайнер лендингов,Москва,От 1 года до 3 лет,Бомбора — крупнейшее в России нон-фикшн издате...,None,"Эксмо, издательство",https://hh.ru/vacancy/76283384,None,65000.0,NaN,RUR
270,73294481,Методист-создатель курсов онлайн изучения Неме...,Москва,От 1 года до 3 лет,Мы активно развиваем онлайн курсы изучения ино...,"[Обучение и развитие, Организаторские навыки, ...",Языковой центр Евразия,https://hh.ru/vacancy/73294481,None,15000.0,30000.0,RUR
583,77152119,Разработчик онлайн-курсов (методолог),Липецк,От 1 года до 3 лет,Команда поддержки бизнеса формирует подразделе...,None,Яндекс,https://hh.ru/vacancy/77152119,None,NaN,NaN,NaN


In [110]:
dt = datetime.datetime.now().strftime("%m-%d-%Y(%H-%M)")
deep_df1.to_csv(f'../data/result_deep_sorted_gamedev_{dt}.csv', index=False)